In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import ylearn
from ylearn.causal_discovery import CausalDiscovery

e:\Application2\anaconda\anaconda\envs\ylearn-py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(2022)

In [3]:
import os
import sys
# 项目根目录
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('./baseline_example.ipynb')))
# 添加系统环境变量
print(BASE_DIR)
sys.path.insert(0, BASE_DIR)

d:\Demo\PythonDemo\2022WAIC_CausalLearing


In [4]:
train = pd.read_csv(BASE_DIR+'/data/train.csv')
test = pd.read_csv(BASE_DIR+'/data/test.csv')

In [5]:
# replace nan
def build_data(train):
    train_ = {}
    for i in train.columns:
        # 对每一序列进行预处理
        # preprocessing for each series
        train_i = train[i]
        if any(train[i].isna()):
            # 对缺失值取均值处理
            print(train[i].isna().sum())
            train_i = train_i.replace(np.nan, train[i].mean())
        if len(train_i.value_counts()) <= 20 and train_i.dtype != object:
            train_i = train_i.astype(int)
        train_[i] = train_i

    return pd.DataFrame(train_)

train = build_data(train)
test = build_data(test)

8
25
2
5


In [6]:
all_cov = list(train.columns)
# save data and their corresponding transformers
class TransData:
    def __init__(self, name, is_obj=False):
        self.is_obj = is_obj
        self.name = name
        self.transformer = None

    def __call__(self, data):
        self.df = data[self.name]
        series = self.df.to_numpy().reshape(-1, 1)
        if self.df.dtype == object:
            self.is_obj = True
            self.transformer = OrdinalEncoder()
            self.data = self.transformer.fit_transform(series).astype(int)
        elif self.df.dtype != int:
            self.transformer = StandardScaler()
            self.data = self.transformer.fit_transform(series)
        else:
            self.data = series

In [7]:
# data preprocessing
data_dict = {}
cat_name = []
test_dict = {}

for name in all_cov:
    t = TransData(name=name)
    t(train)
    data_dict[name] = t.data.reshape(-1, )
    if t.is_obj:
        cat_name.append(name)
    if name not in ['treatment', 'outcome']:
        try:
            test_i = t.transformer.transform(test[name].values.reshape(-1, 1)).reshape(-1, )
        except:
            test_i = test[name]
        test_dict[name] = test_i
train_transformed = pd.DataFrame(data_dict)
test_data = pd.DataFrame(test_dict)
print(cat_name)

['V_8', 'V_10', 'V_14', 'V_26']


In [8]:
train_transformed.head()

,V_0,V_1,V_2,V_3,V_4,V_5,V_6,V_7,V_8,V_9,...,V_32,V_33,V_34,V_35,V_36,V_37,V_38,V_39,treatment,outcome
0,1.723577,-0.305753,-0.713223,-1.621706,-0.110603,0,1.967215,-1.605903,0,-5.175898,...,0.983957,1.170614,-0.043524,1.491432,1.246007,-2,0,3,2,0.965484
1,-0.620006,1.144513,-0.713223,-0.836881,-0.329293,0,-0.321160,0.287543,0,0.193801,...,0.935753,0.229336,0.849727,0.005753,0.958077,0,2,4,0,1.110879
2,-0.844489,0.105237,1.239680,-1.558425,-0.300993,1,-0.277983,0.717924,0,0.193801,...,-2.043339,-0.713962,-0.861334,0.631476,-0.289619,1,1,2,2,-2.258860
3,0.218723,-0.367827,-0.713223,-1.575069,-0.870663,1,0.952558,0.775616,0,0.193801,...,-0.358267,0.035055,0.845040,0.112702,-0.481573,1,0,3,0,-0.267371
4,0.183640,0.928402,-0.713223,-0.134138,0.654154,1,-0.472279,0.776770,0,0.193801,...,-0.078760,-0.046988,-0.110786,0.682046,1.725891,1,0,2,2,-0.166405


In [9]:
'''
V 特征列  x 干预方案 y 作用结果
因果图已知 需根据数据分析 提取各特征列对应的影响变量
构造模型
实现因果预测的模型
'''
V = train_transformed.drop(['treatment', 'outcome'], axis=1).values
x = train_transformed['treatment'].values
y = train_transformed['outcome'].values

In [53]:
# 因果发现 Ylearn自带的没啥用好像 计算速度太慢
slice = np.concatenate((V[:,:5],x.reshape(-1,1),y.reshape(-1,1)),axis=1)
cd = CausalDiscovery(hidden_layer_dim=[3])
est = cd(slice, threshold=0.01, return_dict=True)
print(est)

OrderedDict([(0, [2, 4, 5, 6]), (1, [2, 4, 5, 6]), (2, [4, 6]), (3, [4, 5, 6]), (4, [6]), (5, [2, 4, 6]), (6, [])])


In [45]:
# 因果发现 
CFdata = np.concatenate((V[:,35:40],x.reshape(-1,1),y.reshape(-1,1)),axis=1)

from causallearn.search.ScoreBased.GES import ges
Record = ges(CFdata)
G = Record['G']
# from causallearn.search.ConstraintBased.PC import pc
# G = pc(np.concatenate((V[:,0:20],x.reshape(-1,1),y.reshape(-1,1)),axis=1))
# # visualization using pydot
# cg.draw_pydot_graph()

# from causallearn.search.ConstraintBased.FCI import fci
# G, edges = fci(CFdata)

# # visualization
# from causallearn.utils.GraphUtils import GraphUtils
# pdy = GraphUtils.to_pydot(G)
# pdy.write_png('simple_test_30-40.png')

# from causallearn.search.PermutationBased.GRaSP import grasp
# G = grasp(X=CFdata,score_func='local_score_BIC', maxP=8 )

# # Visualization using pydot
# from causallearn.utils.GraphUtils import GraphUtils
# import matplotlib.image as mpimg
# import matplotlib.pyplot as plt
# import io

# pyd = GraphUtils.to_pydot(G)
# tmp_png = pyd.create_png(f="png")
# fp = io.BytesIO(tmp_png)
# img = mpimg.imread(fp, format='png')
# plt.axis('off')
# plt.imshow(img)
# plt.show()



[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]]
[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [-1  0  0  0  0  0 -1]
 [ 0  0  0  0  0  1  0]]
[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [-1  0  0  0  0  0 -1]
 [ 0  0  1  0  0  1  0]]
[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [-1  0  0  0  0  0 -1]
 [ 0  0  1  0  1  1  0]]
[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0 -1 -1]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 1  0  1  0  0  0 -1]
 [ 0  0  1  0  1  1  0]]
[[ 0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0 -1]
 [ 0  0  0  0  0 -1 -1]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 1  0  1  0  0  0 -1]
 [ 0  1  1 

In [46]:
nodes = G.get_nodes()
print(len(nodes))

def get_G_parents(G, node):
    parent = G.get_parents(node)
    parents = []
    for i in parent:
        parents.append(i.get_name())
    return parents

node_tr = nodes[5]
parents_tr = get_G_parents(G,node_tr)
node_out = nodes[6]
parents_out = get_G_parents(G,node_out)
print(parents_tr)
print(parents_out)

7
['x0', 'x2']
['x1', 'x2', 'x4', 'x5']


In [47]:
score_ges_confounderlist = ['V_1', 'V_2', 'V_3', 'V_6', 'V_7', 'V_11', 'V_13', 'V_18'] + ['V_31','V_33','V_35','V_37']

score_ges_convariatelist = ['V_2', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9', 'V_10', 'V_14', 'V_15', 'V_16', 'V_19'] + ['V_31','V_33','V_36','V_37','V_39']


In [35]:
def x2V_in_parents(parents):
    new_parents = []
    for i in parents:
        new_parents.append(str(i).replace("x","V_"))
    return new_parents

grasp_confounder_list = x2V_in_parents(parents_tr)
grasp_convariate_list = x2V_in_parents(parents_out[:-1])
print(grasp_confounder_list,grasp_convariate_list)

['V_1', 'V_2', 'V_3', 'V_6', 'V_7', 'V_11', 'V_13', 'V_18'] ['V_2', 'V_5', 'V_6', 'V_7', 'V_8', 'V_9', 'V_10', 'V_14', 'V_15', 'V_16', 'V_19']


grasp_confounder_list = ['V_1', 'V_2', 'V_3', 'V_11', 'V_18', 'V_33', 'V_35', 'V_37']

grasp_convariate_list = ['V_1', 'V_2', 'V_3', 'V_4', 'V_9', 'V_10', 'V_11', 'V_14', 'V_16', 'V_17', 'V_18', 'V_19', 'V_21', 'V_22', 'V_28', 'V_29', 'V_30', 'V_31', 'V_32', 'V_33', 'V_35', 'V_36', 'V_39']

In [12]:
'''
训练xmodel的分类器，与ymodel的回归模型
'''
x_model = RandomForestClassifier(n_estimators=150, criterion='entropy', max_features=0.5, max_depth=50)
y_model = RandomForestRegressor(n_estimators=150, max_features=0.5, max_depth=100, )
x_model.fit(V, x)
x_importance = x_model.feature_importances_
y_model_input = np.concatenate((V, x.reshape(-1, 1)), axis=1)
y_model.fit(y_model_input, y=y)
y_importance = y_model.feature_importances_
convariate_list = []
for i, (x_, y_) in enumerate(zip(x_importance, y_importance)):
    if x_ >= 1e-3 and y_ >= 1e-5:
        convariate_list.append(all_cov[i])

In [13]:
# 只考虑有意义的混杂变量
V_new = train_transformed[convariate_list + ['treatment'] + ['outcome']]
print(convariate_list)

['V_0', 'V_1', 'V_2', 'V_3', 'V_4', 'V_5', 'V_6', 'V_7', 'V_10', 'V_11', 'V_12', 'V_13', 'V_14', 'V_15', 'V_17', 'V_18', 'V_19', 'V_21', 'V_22', 'V_23', 'V_24', 'V_25', 'V_28', 'V_29', 'V_30', 'V_31', 'V_32', 'V_33', 'V_34', 'V_35', 'V_36', 'V_37', 'V_38', 'V_39']


In [34]:
# 重定义混淆因子
confounder_list = ['V_1','V_3','V_4','V_5','V_7','V_6','V_13','V_14','V_34','V_35','V_36']
# confounder_list = ['V_1','V_13','V_14']
for x in confounder_list:
    convariate_list.remove(x) 
print(convariate_list)



['V_0', 'V_2', 'V_10', 'V_11', 'V_12', 'V_15', 'V_17', 'V_18', 'V_19', 'V_21', 'V_22', 'V_23', 'V_24', 'V_25', 'V_28', 'V_29', 'V_30', 'V_31', 'V_32', 'V_33', 'V_37', 'V_38', 'V_39']


In [11]:
def ce2csv(ce_total, file_name="result.csv"):
    pd.DataFrame(ce_total).to_csv(BASE_DIR + "/results/" + file_name,
                              index=False,
                              header=['ce_1', 'ce_2'])
    return 


# 评估训练集得分
def get_nrmse(predictions, targets):
    differences = predictions - targets
    differences_square = differences**2
    mean_of_differences_square = differences_square.mean()
    rmse = np.sqrt(mean_of_differences_square)
    nrmse = rmse / (targets.mean())
    return np.abs(nrmse)

def ce_score(ce, V_new):
    '''
    !!!  无效，评分为因果效应评分，并非直接减去outcome
    '''
    TREATMENT = 1
    mask = V_new['treatment']== TREATMENT
    train_1_nrmse = get_nrmse(ce[mask, TREATMENT-1], V_new['outcome'][mask])
    TREATMENT = 2
    mask = V_new['treatment']== TREATMENT
    train_2_nrmse = get_nrmse(ce[mask, TREATMENT-1], V_new['outcome'][mask])
    print(train_1_nrmse,train_2_nrmse)
    return train_1_nrmse,train_2_nrmse


def get_ce(data, x1_model, x2_model):
    ce1 = x1_model.estimate(data)
    ce2 = x2_model.estimate(data)
    return np.concatenate([ce1.reshape(-1, 1), ce2.reshape(-1, 1)], axis=1)

In [81]:
train_transformed.groupby('treatment').outcome.mean()

treatment
0   -0.302371
1   -0.381580
2    0.236624
Name: outcome, dtype: float64

In [77]:
from ylearn.estimator_model import TLearner, XLearner,SLearner
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor
from sklearn import svm
import xgboost as xgb
model = xgb.XGBRegressor(seed=1850,n_estimators=1000,learning_rate=0.1,
                         max_depth=11)
# model = RandomForestRegressor(n_estimators=150)
# from model import Regressor
# import imp
# imp.reload(Regressor)
# model = GradientBoostingRegressor(learning_rate=0.01, criterion="friedman_mse",
#                                              n_estimators=150, max_features=0.6)
tl1 = XLearner(model=model)
tl2 = XLearner(model=model)
tl1.fit(data=train_transformed, treatment='treatment', outcome='outcome', treat=1,
        control=0, covariate=grasp_confounder_list)
tl2.fit(data=train_transformed, treatment='treatment', outcome='outcome', treat=2,
        control=0, covariate=grasp_confounder_list)
print(tl1.estimate(quantity="CATE"))
print(tl2.estimate(quantity="CATE"))


0.3977107
0.7562739


ce是训练集（train.csv）上的因果效应，另外需要估计测试集（test.csv）上的因果效应ce_test。最后需要把ce_test拼接在ce之后，存储到一个csv文件中上传到平台取得得分。得分为一个数值，越小说明结果越接近真实值，得分最小值为0。

In [79]:
ce = get_ce(train_transformed, tl1, tl2)
ce_test = get_ce(test_data, x1_model=tl1, x2_model=tl2)
ce_total = np.concatenate((ce, ce_test), axis=0)
ce2csv(ce_total,"GraspConfounder_Xlearn_XGB_n1000_depth11.csv")


以下内容是给出的其他两种方法

In [19]:
from ylearn.estimator_model import DML4CATE

dml = DML4CATE(cf_fold=1, x_model=RandomForestClassifier(n_estimators=250, criterion="entropy", max_depth=150, min_samples_leaf=2, min_samples_split=3, max_features=3),
               y_model=RandomForestRegressor(n_estimators=250, max_depth=150, min_samples_leaf=2, min_samples_split=2, max_features=3), is_discrete_treatment=True)
dml.fit(data=V_new, outcome='outcome', treatment='treatment', covariate=confounder_list,)
ce_dml = dml.effect_nji(data=V_new, control=0)
ce_dml_test = dml.effect_nji(data=test_data, control=0)
ce_dml_train = ce_dml[:, :, 1:].reshape(-1, 2)
ce_dml_all = np.concatenate([ce_dml_train, ce_dml_test[:, :, 1:].reshape(-1, 2)], axis=0)

In [35]:
from ylearn.estimator_model import CausalTree

ct1 = CausalTree(max_depth=100, min_samples_split=2, max_features=20)
ct2 = CausalTree(max_depth=100, max_features=20)
ct1.fit(data=V_new, outcome='outcome', treatment='treatment',adjustment=confounder_list, 
                covariate=convariate_list, treat=[1], control=[0])
ct2.fit(data=V_new, outcome='outcome', treatment='treatment',adjustment=confounder_list, 
                covariate=convariate_list, treat=[2], control=[0])

CausalTree(max_depth=100, max_features=20)

In [95]:
ce_ct = get_ce(V_new, ct1, ct2)
ce_ct_test = get_ce(test_data, ct1, ct2)
ce_ct_all = np.concatenate([ce_ct, ce_ct_test], axis=0)
ce2csv(ce_ct_all,"CausalTree.csv")

In [36]:
from ylearn.estimator_model.doubly_robust import DoublyRobust
from sklearn.ensemble import GradientBoostingRegressor
n = V_new.shape[0]
dr = DoublyRobust(
    x_model=RandomForestClassifier(n_estimators=100, max_depth=100, min_samples_leaf=int(n/100)),
    y_model=GradientBoostingRegressor(n_estimators=100, max_depth=100, min_samples_leaf=int(n/100)),
    yx_model=GradientBoostingRegressor(n_estimators=100, max_depth=100, min_samples_leaf=int(n/100)),
    cf_fold=1,
    random_state=2022,
)
dr.fit(data=V_new, outcome="outcome", treatment="treatment",control=0, adjustment=confounder_list, covariate=convariate_list,)

DoublyRobust(x_model=RandomForestClassifier(max_depth=100, min_samples_leaf=361), y_model=GradientBoostingRegressor(max_depth=100, min_samples_leaf=361), yx_model=GradientBoostingRegressor(max_depth=100, min_samples_leaf=361), random_state=2022)

In [38]:
dr_pred_2 = dr.estimate(data=test_data, all_tr_effects=True).reshape(-1,3)
dr_pred_1 = dr.estimate(data=V_new, all_tr_effects=True).reshape(-1,3)
dr_ce_1 = dr_pred_1[:,1:]
dr_ce_2 = dr_pred_2[:,1:]
dr_ce_total = np.concatenate((dr_ce_1,dr_ce_2),axis=0)
ce2csv(dr_ce_total,"handcrafted_DoublyRobust2.csv")